<a href="https://colab.research.google.com/github/mehulmotani/share/blob/main/logistic_iotfailure_prediction-cleaningup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IoT Equipment Failure Prediction using Sensor data
## 1 Environment Setup
### 1.1 Import dependent libraries

In [16]:
# Import libraries
import pandas as pd
import numpy as np
import pdb
import json
import re
import requests
import sys
import types

In [17]:
# Import libraries
from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

## 2 Create IoT Predictive Analytics Functions

In [18]:
# Function to extract Column names of dataset
def dataset_columns(dataset):
    return list(dataset.columns.values)

# Function to train Logistic regression model
def train_logistic_regression(x_vals, y_vals):
    logistic_regression_model = LogisticRegression()
    logistic_regression_model.fit(x_vals, y_vals)
    return logistic_regression_model

# Function to return Predicted values
def score_data(trained_model, x_vals):
    ypredict = trained_model.predict(x_vals)
    return ypredict

# Function to calculate Prediction accuracy of model
def model_accuracy(trained_model, variables, targets):
    accuracy_score = trained_model.score(variables, targets)
    return accuracy_score

# Function to generate Confusion matrix
def confusion_matrix(actfail, predictfail):
  # Compute Confusion matrix
  print("Actual, Predicted Observations:  ",len(actfail), len(predictfail))
  # print(actfail, predictfail)
  anpn = 0
  anpy = 0
  aypn = 0
  aypy = 0
  
  for i in range(len(actfail)):
      if (actfail[i]==0 and predictfail[i]==0):
          anpn = anpn + 1
      elif (actfail[i]==0 and predictfail[i]==1):
          anpy = anpy + 1
      elif (actfail[i]==1 and predictfail[i]==0):
          aypn = aypn + 1
      else:
          aypy = aypy + 1
  # Confusoin matrix
  print ("--------------------------------------------")
  print ("Confusion Matrix")
  print ("--------------------------------------------")
  print ("              ", "Predicted N", "Predicted Y")
  print ("Actual N      ", anpn,"          ", anpy) 
  print ("Actual Y      ", aypn,"          ", aypy)
  print ("--------------------------------------------")
  print ("Total observations  :  ", anpn+anpy+aypn+aypy)
  print ("False Positives     :  ", anpy)
  print ("False Negatives     :  ", aypn)
  print ("Overall Accuracy    :  ", round((float(anpn+aypy)/float(anpn+anpy+aypn+aypy))*100, 2), "%")
  print ("Sensitivity/Recall  :  ", round((float(aypy)/float(aypn+aypy))*100, 2), "%")
  print ("Specificity         :  ", round((float(anpn)/float(anpn+anpy))*100, 2), "%")
  print ("Precision           :  ", round((float(aypy)/float(anpy+aypy))*100, 2), "%")
  print ("--------------------------------------------")


## 4 Read IoT Sensor data from database

In [29]:
#df_iotdata = pd.read_csv("iot_sensor_dataset.csv")
df_iotdata = pd.read_csv("https://raw.githubusercontent.com/mehulmotani/share/main/iot_sensor_dataset_clean.csv")

url="https://raw.githubusercontent.com/mehulmotani/share/main/tennis.csv"
df_tennis=pd.read_csv(url)
df_tennis.head()

,PlayTennis,Outlook,Temperature,Humidity,Wind
0,no,sunny,hot,high,weak
1,no,sunny,hot,high,strong
2,yes,overcast,hot,high,weak
3,yes,rain,mild,high,weak
4,yes,rain,cool,normal,weak


In [30]:
print ("Number of Observations :", len(df_iotdata))
df_iotdata.head()

Number of Observations : 944


,footfall,atemp,selfLR,ClinLR,DoleLR,PID,outpressure,inpressure,temp,fail
0,0,7,7,1,6,6,36,3,1,1
1,190,1,3,3,5,1,20,4,1,0
2,31,7,2,2,6,1,24,6,1,0
3,83,4,3,4,5,1,28,6,1,0
4,640,7,5,6,4,0,68,6,1,0


In [21]:
# Print dataset column names
datacolumns = dataset_columns(df_iotdata)
print ("Data set columns : ", list(datacolumns))
v_feature_list = ['atemp', 'PID', 'outpressure', 'inpressure', 'temp']
v_target = ['fail']

Data set columns :  ['footfall', 'atemp', 'selfLR', 'ClinLR', 'DoleLR', 'PID', 'outpressure', 'inpressure', 'temp', 'fail']


## 5 Run Failure Prediction algorithm on IoT data
### 5.1 Split data into Training and Test data

In [22]:
# Split Training and Testing data
train_x, test_x, train_y, test_y = train_test_split(df_iotdata[v_feature_list], df_iotdata[v_target], train_size=0.8, test_size=0.2)
print ("Train x counts : ", len(train_x), len(train_x.columns.values))
print ("Train y counts : ", len(train_y))
 
print ("Test x counts : ", len(test_x), len(test_x.columns.values))
print ("Test y counts : ", len(test_y))


Train x counts :  755 5
Train y counts :  755
Test x counts :  189 5
Test y counts :  189


### 5.2 Train the Predictive model
### $\ln(\frac{p}{1-p}) = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \beta_5 x_5$

In [23]:
# Training Logistic regression model
#trained_logistic_regression_model = train_logistic_regression(train_x, train_y)
trained_logistic_regression_model = train_logistic_regression(train_x, train_y.values.ravel())

train_accuracy = model_accuracy(trained_logistic_regression_model, train_x, train_y)

# Testing the logistic regression model
test_accuracy = model_accuracy(trained_logistic_regression_model, test_x, test_y)

print ("Training Accuracy : ", round(train_accuracy * 100, 2), "%")



Training Accuracy :  90.6 %


### 5.3 Score the Test data using the Trained model

In [24]:
# Model accuracy: Score and construct Confusion matrix for Test data
actfail = test_y.values
predictfail = score_data(trained_logistic_regression_model, test_x)


## 6 Confusion matrix for deeper analysis of Prediction accuracy
#####   Confusion matrix outputs below can be used for calculating more customised Accuracy metrics

In [25]:
# Print Count of Actual fails, Predicted fails
# Print Confusion matrix
confusion_matrix(actfail, predictfail)

Actual, Predicted Observations:   189 189
--------------------------------------------
Confusion Matrix
--------------------------------------------
               Predicted N Predicted Y
Actual N       102            13
Actual Y       5            69
--------------------------------------------
Total observations  :   189
False Positives     :   13
False Negatives     :   5
Overall Accuracy    :   90.48 %
Sensitivity/Recall  :   93.24 %
Specificity         :   88.7 %
Precision           :   84.15 %
--------------------------------------------
